In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from ortools.linear_solver import pywraplp
import timeit
import random
import time

In [8]:
def model(packages,cargo):
    solver = pywraplp.Solver('Model2DR', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # importo dimensioni dei pacchi e del camion
    n=len(packages)

    w =[packages[i].getW() for i in range(n)]
    d =[packages[i].getD() for i in range(n)]

    print(w)
    print(d)

    W =cargo.getW()
    D =solver.IntVar(0,solver.infinity(),"D")

    Md=sum(d)
    Mw=W+min(w)

    # definisco le variabili
    s1 =[[solver.BoolVar("s1%d%d" % (i,j)) for i in range(n)] for j in range(n)]
    s2 =[[solver.BoolVar("s2%d%d" % (i,j)) for i in range(n)] for j in range(n)]
    s3 =[[solver.BoolVar("s3%d%d" % (i,j)) for i in range(n)] for j in range(n)]
    s4 =[[solver.BoolVar("s4%d%d" % (i,j)) for i in range(n)] for j in range(n)]

    x =[solver.IntVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
    y =[solver.IntVar(0,solver.infinity(),"y%d" % i) for i in range(n)]



    # definisco i constraints
    for i in range(n):
        for j in range(n):
            if(i < j):
                solver.Add(x[i] + w[i] <= x[j] + Mw*(1-s1[i][j]))           #(1)
                solver.Add(y[i] + d[i] <= y[j] + Md*(1-s2[i][j]))           #(2)
                solver.Add(x[j] + w[j] <= x[i] + Mw*(1-s3[i][j]))           #(3)
                solver.Add(y[j] + d[j] <= y[i] + Md*(1-s4[i][j]))           #(4)
                solver.Add(s1[i][j]+s2[i][j]+s3[i][j]+s4[i][j]>=1)          #(5)
        solver.Add(x[i] + w[i] <= W)                   #(6)
        solver.Add(y[i] + d[i] <= D)                   #(7)

    #funzione obiettivo
    objective = solver.Objective()
    objective.SetCoefficient(D,1)
    objective.SetMinimization()

    #soluzione
    solver.Solve()

    #lista soluzioni
    s = []
    for i in range(n):
        s.append([x[i].solution_value(), y[i].solution_value(), i, 0])
    return s

SyntaxError: invalid syntax (<ipython-input-8-33174b67ae27>, line 52)

In [9]:
def plotSolution(solution, camion, packages):
    boxes = []

    width = [p.dimensioni[0] for p in packages]
    depth = [p.dimensioni[1] for p in packages]
    print width
    #bin
    container = Rectangle((0,0), camion.dimensioni[1], camion.dimensioni[0], facecolor='white')
    boxes.append(container)

    #items
    for a in solution:
        w =width[a[2]] if a[3] == 0 else depth[a[2]]
        d =depth[a[2]] if a[3] == 0 else width[a[2]]
        box = Rectangle((a[1],a[0]), d, w, facecolor='r')
        boxes.append(box)
    #plot
    facecolors = [b.get_facecolor() for b in boxes]

    collection = PatchCollection(boxes, facecolor=facecolors, alpha=0.4, edgecolor='black', linewidths=2)

    ax = plt.gca(xlabel='depth',ylabel='width')
    ax.add_collection(collection)
    ax.autoscale_view()
    ax.axis('equal')
    plt.draw_if_interactive()

In [4]:
def main():
    packages =[]
    for i in range(0,5):
        w=120
        d=80
        packages.append(box([w,d]))
    camion =cargo([250,1350])
    t=time.time()
    s =model(packages,camion)
    print s
    plotSolution(s,camion,packages)
    t=time.time()-t
    print(t)

In [5]:
class box:
    def __init__(self,dim):
        self.dimensioni=dim

    def getW(self):
        return self.dimensioni[0]

    def getD(self):
        return self.dimensioni[1]


In [6]:
class cargo:
    def __init__(self,dim):
        self.dimensioni=dim

    def getW(self):
        return self.dimensioni[0]

    def getD(self):
        return self.dimensioni[1]


In [7]:
main()

[120, 120, 120, 120, 120]
[80, 80, 80, 80, 80]


NameError: global name 'r' is not defined